In [5]:
# Import the packages
from numpy import repeat
from ydata.synthesizers.regular import RegularSynthesizer
from ydata.connectors import LocalConnector
from ydata.connectors.filetype import FileType

In [7]:
# Read the data (Only if you want to train a new network)
# Initialize the YData's connector
connector = LocalConnector()

# Read the data
data = connector.read_file('data_processed.csv', file_type = FileType.CSV)

In [8]:
# Initialize and Load an alerady trained Synthesizer.
network_synth = RegularSynthesizer(filter_outliers=False)
network_synth = network_synth.load('trained_model.pkl')

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator BayesianRidge from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator BayesianGaussianMixture from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own 

In [9]:
# In case you want to train a new network.
#network_synth.fit(data)

In [12]:
# Create the samples and flag them.
synth_sample = network_synth.sample(5000).to_pandas()
synth_sample.index = repeat(999999, len(synth_sample))

INFO: 2022-02-17 14:45:49,641 [SYNTHESIZER] - Start generating model samples.


In [17]:
synth_sample.columns = data.to_pandas().columns

In [18]:
connector.write_file(data= synth_sample, path='data_sampled.csv', index=False)